# Random Forest

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics

from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
boston = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
#boston

# EDA

In [3]:
data = np.hstack([boston.values[::2, :], boston.values[1::2, :3]])
#target = boston.values[1::2, 2]
#data
#Convert array into DataFrame and rename columns
boston_data = pd.DataFrame(data, columns =['crime', 'land_zone', 'indus', 'C_River', 'nox','avg_rooms', 'age','dist','r_highway', 'tax', 'pt_ratio','blacks','lstat','price_median_value'])
#boston_data = pd.DataFrame(target, columns =['price_Median_value'])
#boston_data.shape

#Dropping Duplicates
boston_data.duplicated().sum()
#boston_data.drop_duplicates()
#boston_data.shape
#Check Missing Values
boston_data.info()
#Check Null Values
boston_data.isna().sum()
boston_data.isnull().sum()
#boston_data.notna().sum()
#boston_data.notnull().sum()
#boston_data.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   crime               506 non-null    float64
 1   land_zone           506 non-null    float64
 2   indus               506 non-null    float64
 3   C_River             506 non-null    float64
 4   nox                 506 non-null    float64
 5   avg_rooms           506 non-null    float64
 6   age                 506 non-null    float64
 7   dist                506 non-null    float64
 8   r_highway           506 non-null    float64
 9   tax                 506 non-null    float64
 10  pt_ratio            506 non-null    float64
 11  blacks              506 non-null    float64
 12  lstat               506 non-null    float64
 13  price_median_value  506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


crime                 0
land_zone             0
indus                 0
C_River               0
nox                   0
avg_rooms             0
age                   0
dist                  0
r_highway             0
tax                   0
pt_ratio              0
blacks                0
lstat                 0
price_median_value    0
dtype: int64

# Check Collineariry/Dependability of Variabels

In [27]:
#colinearity check
boston_data.corr()  
#Format the plot background and scatter plots for all the variables
#sns.set_style(style='darkgrid') 
#sns.pairplot(boston_data) #colinearity check

,crime,land_zone,indus,C_River,nox,avg_rooms,age,dist,r_highway,tax,pt_ratio,blacks,lstat,price_median_value
crime,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
land_zone,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
indus,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
C_River,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
nox,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
avg_rooms,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
age,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
dist,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
r_highway,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
tax,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


# Remove outliers

In [179]:
#1. remove outlier using median imputation

#plt.figure(figsize=(17,1))
for var in range(0,boston_data.shape[1]):
    temp=boston_data.columns[var]
    lower_quant=boston_data[temp].quantile(0.25)
    upper_quant=boston_data[temp].quantile(0.75)
    med=boston_data[temp].median()
    #print("\nColumn :",temp)
    #print("Lower Quantile:",lower_quant, "Upper Quantile :",upper_quant,"Median :",med)
    
    boston_data[temp] = np.where( (boston_data[temp] <lower_quant), med , boston_data[temp]) 
    boston_data[temp] = np.where( (boston_data[temp] >upper_quant), med , boston_data[temp])
    #print("\n",boston_data.columns[var])
    #sns.boxplot(data=boston_data[temp])
   # plt.show()
        
#After Outlier Median imputation 
boston_clean=boston_data


In [152]:
df=boston_data

# 2. Z-Score treatment for outlier removal

In [4]:
#Find Boundary Values
df=boston_data
for var in range(0,df.shape[1]):
    #if (df.columns[var].dtype != "category"):
    lower_limit= df[df.columns[var]].mean() - 3 * df[df.columns[var]].std()
    upper_limit=df[df.columns[var]].mean() + 3*df[df.columns[var]].std()
    #print("Highest allowed",lower_limit)
    #print("Lowest allowed",uppler_limit)
    #find Outliers trim it
    #new_df =df[(df['crime'] > lower_limit) | (df['crime'] < uppler_limit)]
    #new_df 
    #Capping on Outliers
    lower_limit,upper_limit
    df[df.columns[var]] = np.where(df[df.columns[var]]>upper_limit, upper_limit,df[df.columns[var]])
    df[df.columns[var]] = np.where(df[df.columns[var]]<lower_limit, lower_limit,df[df.columns[var]])
boston_data = df   

In [180]:
boston_data.describe()

,crime,land_zone,indus,C_River,nox,avg_rooms,age,dist,r_highway,tax,pt_ratio,blacks,lstat,price_median_value
count,5.060000e+02,506.0,506.000000,506.0,5.060000e+02,5.060000e+02,506.0,5.060000e+02,506.000000,506.000000,506.000000,5.060000e+02,5.060000e+02,5.060000e+02
mean,2.565100e-01,0.0,12.089091,0.0,5.380000e-01,6.208500e+00,77.5,3.207450e+00,10.217391,433.047431,19.404842,3.914400e+02,1.136000e+01,2.120000e+01
std,2.278210e-15,0.0,3.661216,0.0,4.556419e-15,4.534193e-14,0.0,3.689588e-14,8.227951,141.851162,0.513618,1.820789e-12,6.401213e-14,7.468082e-14
min,2.565100e-01,0.0,9.690000,0.0,5.380000e-01,6.208500e+00,77.5,3.207450e+00,5.000000,330.000000,19.050000,3.914400e+02,1.136000e+01,2.120000e+01
25%,2.565100e-01,0.0,9.690000,0.0,5.380000e-01,6.208500e+00,77.5,3.207450e+00,5.000000,330.000000,19.050000,3.914400e+02,1.136000e+01,2.120000e+01
50%,2.565100e-01,0.0,9.690000,0.0,5.380000e-01,6.208500e+00,77.5,3.207450e+00,5.000000,330.000000,19.050000,3.914400e+02,1.136000e+01,2.120000e+01
75%,2.565100e-01,0.0,18.100000,0.0,5.380000e-01,6.208500e+00,77.5,3.207450e+00,24.000000,666.000000,20.200000,3.914400e+02,1.136000e+01,2.120000e+01
max,2.565100e-01,0.0,18.100000,0.0,5.380000e-01,6.208500e+00,77.5,3.207450e+00,24.000000,666.000000,20.200000,3.914400e+02,1.136000e+01,2.120000e+01


# Split dataset

In [6]:
#Split Data into X and y
array = boston_data.values
X = array[:, 0:13]
y = array[:, 13]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((339, 13), (339,), (167, 13), (167,))

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X=sc.fit_transform(X)


In [8]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y) 

RandomForestRegressor(random_state=0)

In [9]:
#predicting on the test set and creating submission file

y_pred = regressor.predict(X_test)
print(y_pred)

data_pred=pd.DataFrame(data=np.array([[0.25651,0.0,9.69,0.0,0.538,6.5750,65.2,4.09000,5.0,296.0,19.05,391.44,11.36]]),columns=['crime','land_zone','indus','C_River','nox','avg_rooms','age','dist','r_highway','tax','pt_ratio','blacks','lstat'])

Y_pred = regressor.predict(data_pred)  # test the output by changing values
print("Predicted value :",Y_pred)
accuracy = regressor.score(X,y)
print( 'Random Forest Accuracy: ', accuracy*100,'%')
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

[24.246 12.578 27.614 19.291 13.833 24.043 20.449 32.154 12.888 35.683
 15.062 13.951 33.321 25.786 19.794 22.063 31.114 20.713 18.255 14.239
 15.113 21.554 44.377  8.932 23.093 23.891 16.668 18.042 18.028 16.839
 18.046 29.588 43.28  15.582  9.503 47.088 46.239 15.411 25.248 17.656
 26.066 21.867 20.506 28.082 14.431 18.476 16.37  13.951 19.194 31.278
 25.697 34.793 45.594 16.924 25.559 18.452 47.707 14.349 14.702 10.68
 13.777  8.575 21.774 27.515  7.673 20.814 20.278 13.423 22.348 20.07
 19.881 16.025 24.199 31.652 16.093 49.352 46.746 16.342 22.064 11.811
 19.363 14.826 23.417 19.692 16.007 20.27  22.332 18.03  20.743 19.201
 18.802 24.77  20.889 22.808 20.908 19.049 24.002 30.998 11.933 16.017
 23.408 45.792 10.437 20.675 18.92  35.121 20.771 34.477 24.385 19.832
 21.387 16.558 34.391 23.249 26.453 29.905 27.374 19.135 19.328 13.495
 19.027 13.302 23.293 48.905 14.468 22.014 25.695 22.532 44.014 18.886
 19.013 19.195 11.91  16.35  31.234 20.316 20.026 35.271 22.489 26.956
 22.281 

In [35]:
boston_data

,crime,land_zone,indus,C_River,nox,avg_rooms,age,dist,r_highway,tax,pt_ratio,blacks,lstat,price_median_value
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
